* Tech crunch : 
    - __Fundings__         -
* Snow : 
    - __Contact details__  -
* angel.co : 
    - __jobs__            - Done
    - __tech_req__        - Done
    - __compensation__    - Done
    - __number_of_emp__   - Done
    - __location__        - Done
    - __area of interest__- Done
    - __Investors__       - Done
    - __Founder__         - Done


In [33]:
import cfscrape
from bs4 import BeautifulSoup
import re
import requests
from googlesearch import search 

def get_urls(comapny_name):
    sites = ['LinkedIn', 'angel.co jobs', 'techcrunch', 'snow data']
    urls = {}
    for site in sites:
        query = comapny_name + site

        url_generator = search(query, tld="com", num=1, stop=1, pause=2)
        for url in url_generator:
            urls[site] = url
    return urls

def break_protection(link):
    cfscrape.DEFAULT_CIPHERS = 'TLS_AES_256_GCM_SHA384:ECDHE-ECDSA-AES256-SHA384'
    scraper = cfscrape.create_scraper()
    data = scraper.get(link).content
    soup = BeautifulSoup(data, 'html.parser')
    return soup

def get_angel_co_data(soup):
    links = []
    for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
        links.append(link.get('href'))

    jobs = soup.find_all(class_='u-fontSize18')
    job = [i.text for i in jobs if i.text != '']
    all_job_categories = [i for i in job if '\n' in i]

    # Get jobs
    dict_1 = {}
    value = []
    for i in range(len(all_job_categories)):

        if all_job_categories[i].strip() == 'Software Engineering Jobs':
            value.extend(job[job.index(all_job_categories[i])+1 : job.index(all_job_categories[i+1])])

        elif all_job_categories[i].strip() == 'Other Jobs':
            value.extend(job[job.index(all_job_categories[i])+1:])

    dict_1['jobs'] = value

    # Get links
    check_list = []
    for name in value:
        val = " ".join(name.lower().replace('-','').strip().split()).replace(' ','-').replace('(','').replace(')','')
        check_list.append(val.replace('.',''))

    job_links = []
    for name in check_list:
        job_links.extend([i for i in links if name in i])
    job_links = list(set(job_links))

    # Go to each link and get tech_requirments, employee number and compensation
    tech_req = []
    Compensation = []
    c = 0
    for link in job_links:
        scraper_1 = cfscrape.create_scraper()
        data_1 = scraper_1.get(link).content
        soup_1 = BeautifulSoup(data_1, 'html.parser')

        inf = soup_1.find_all(class_='s-grid--preMd1')
        list_info = [i.text.strip().split('\n') for i in inf]

        for_tech_req = [i for i in list_info[0] if i != '']
        for_comp_info = [i for i in list_info[1] if i != '']

        if 'Skills' in for_tech_req and c != 0:
            tech_req.extend(tech_req + for_tech_req[for_tech_req.index('Skills')+1 : for_tech_req.index('Compensation')])
        elif 'Skills' in for_tech_req:
            tech_req.extend(for_tech_req[for_tech_req.index('Skills')+1 : for_tech_req.index('Compensation')])
        c += 1
        if 'Compensation' in for_tech_req:
            Compensation.extend([for_tech_req[for_tech_req.index('Compensation')+1]])

        key = 'Employees'
        if key not in dict_1.keys():
            dict_1['Employees'] = [i.replace(' Employees','') for i in for_comp_info if 'Employees' in i]

    dict_1['Compensation'] = Compensation
    dict_1['tech_req'] = set(tech_req)
    
    # Get location and area of interest
    soup_1 = break_protection(urls['angel.co jobs'].replace('/jobs', ''))
    info = [i.text for i in soup_1.findAll(class_='tag')]
    dict_1['Location'] = info[0]
    dict_1['Area of interest'] = info[1:]
    
    #Get inverstors
    dict_1['investors'] = [i.text for i in soup_1.findAll(class_='startup-link') if i.text != '' ]
    
    # GetFounders
    founders = [i.text.strip().split('\n\n\n') for i in soup_1.findAll(class_='founders section') if i.text.strip().split('\n\n\n') != '' ]
    founders_list = [j for j in founders[0] if j != '']
    for i in range(len(founders_list)):
        dict_1['Founder_' + str(i)] = founders_list[i]
    return dict_1


In [34]:
comapny_name = 'Think Analytics'
urls = get_urls(comapny_name)
soup = break_protection(urls['angel.co jobs'])
dict_1 = get_angel_co_data(soup)

In [35]:
dict_1

{'jobs': ['Frontend Associate - HTML/CSS/Javascript',
  'Backend/Programmer Associate - Python/Java',
  'Hybrid Mobile Development',
  'Native Android Developer',
  'Senior Azure Cloud Solution Architect'],
 'Employees': ['51-200'],
 'Compensation': ['₹4L – ₹7L', '₹5L – ₹8L', '₹12L – ₹15L'],
 'tech_req': {'Android, SQLite, Android SDK',
  'Microsoft Azure',
  'UI/UX Design, Cordova, Ionic Framework'},
 'Location': 'Mumbai',
 'Area of interest': ['Analytics',
  'Big Data Analytics',
  'Business Analytics',
  'Predictive Analytics'],
 'investors': [],
 'Founder_0': 'Suryadip Ghoshal\n\nFounder\nFounder @Think Analytics • Worked at @pwc-consulting',
 'Founder_1': 'Amit Das\n\nFounder'}